In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory
from collections import Counter



In [3]:
##################################################
# Load members and
##################################################
demographics = pd.read_csv('../../input/preprocessed_data/demographics.csv')


In [18]:
##################################################
# Convert string to datetime format
##################################################
demographics['registration_init_time']  = demographics.registration_init_time.apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))


In [20]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6769473 entries, 0 to 6769472
Data columns (total 12 columns):
msno                      object
city                      int8
bd                        int8
gender                    int8
registered_via            int8
registration_init_time    datetime64[ns]
is_registerd_via_4        int8
is_registerd_via_7        int8
bd_zone                   int8
is_living_in_city-21      int8
is_living_in_city-2       int8
city_zone                 int8
dtypes: datetime64[ns](1), int8(10), object(1)
memory usage: 167.9+ MB


In [21]:
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(demographics)


reduce memory


100%|██████████| 12/12 [00:00<00:00, 71.47it/s]


In [22]:
demographics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6769473 entries, 0 to 6769472
Data columns (total 12 columns):
msno                      object
city                      int8
bd                        int8
gender                    int8
registered_via            int8
registration_init_time    datetime64[ns]
is_registerd_via_4        int8
is_registerd_via_7        int8
bd_zone                   int8
is_living_in_city-21      int8
is_living_in_city-2       int8
city_zone                 int8
dtypes: datetime64[ns](1), int8(10), object(1)
memory usage: 167.9+ MB


In [7]:
demographics.head()

,msno,city,bd,gender,registered_via,registration_init_time,is_registerd_via_4,is_registerd_via_7,bd_zone,is_living_in_city-21,is_living_in_city-2,city_zone
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,29,2,1,20110911,0,0,3,0,0,1
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,29,2,2,20110914,0,1,3,0,0,1
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,29,2,1,20110915,0,0,3,0,0,1
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,29,2,1,20110915,0,0,3,0,0,1
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,0,3,20110915,0,0,3,0,0,3


In [9]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']] # we do not need is_churn


In [23]:
df = pd.merge(train, demographics, on = 'msno',how = 'left')

In [14]:
df.columns

Index(['msno', 'city', 'bd', 'gender', 'registered_via',
       'registration_init_time', 'is_registerd_via_4', 'is_registerd_via_7',
       'bd_zone', 'is_living_in_city-21', 'is_living_in_city-2', 'city_zone'],
      dtype='object')

In [29]:
df

,msno,city,bd,gender,registered_via,registration_init_time,is_registerd_via_4,is_registerd_via_7,bd_zone,is_living_in_city-21,is_living_in_city-2,city_zone
0,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13.0,18.0,1.0,4.0,2013-12-27,0.0,0.0,1.0,0.0,0.0,2.0
1,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,29.0,2.0,2.0,2014-01-09,0.0,1.0,3.0,0.0,0.0,1.0
2,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13.0,35.0,0.0,2.0,2014-01-25,0.0,1.0,4.0,0.0,0.0,2.0
3,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,22.0,29.0,2.0,4.0,2014-01-26,0.0,0.0,3.0,0.0,0.0,3.0
4,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1.0,29.0,2.0,2.0,2014-01-29,0.0,1.0,3.0,0.0,0.0,1.0
5,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,13.0,21.0,0.0,3.0,2014-02-12,0.0,0.0,2.0,0.0,0.0,2.0
6,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,13.0,29.0,2.0,2.0,2014-02-28,0.0,1.0,3.0,0.0,0.0,2.0
7,dfLS2/Pom6O3iUpo+mf/K1Bc6g+7LFKzoNf+zlJtDoQ=,1.0,29.0,2.0,2.0,2014-03-23,0.0,1.0,3.0,0.0,0.0,1.0
8,a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=,13.0,32.0,2.0,2.0,2014-03-24,0.0,1.0,3.0,0.0,0.0,2.0
9,f91n3lDipDjRtAVNgMGnTsxP0VrzDsfB0Hdth12gqQE=,1.0,29.0,2.0,2.0,2014-04-20,0.0,1.0,3.0,0.0,0.0,1.0


In [52]:
now_time = datetime.strptime('2017-03-01', '%Y-%m-%d') 
df['how_long_has_benn_a_memmbership_of_kkbox_days'] = [(now_time - datetime.utcfromtimestamp(r_i_t.tolist()/1e9)).days if pd.notnull(r_i_t) else -1 for r_i_t in df.registration_init_time.values]
df['how_long_has_benn_a_memmbership_of_kkbox_years'] = df.how_long_has_benn_a_memmbership_of_kkbox_days / 360

In [53]:
df

,msno,city,bd,gender,registered_via,registration_init_time,is_registerd_via_4,is_registerd_via_7,bd_zone,is_living_in_city-21,is_living_in_city-2,city_zone,how_long_has_benn_a_memmbership_of_kkbox,how_long_has_benn_a_memmbership_of_kkbox_days,how_long_has_benn_a_memmbership_of_kkbox_years
0,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13.0,18.0,1.0,4.0,2013-12-27,0.0,0.0,1.0,0.0,0.0,2.0,1160,1160,3.222222
1,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,29.0,2.0,2.0,2014-01-09,0.0,1.0,3.0,0.0,0.0,1.0,1147,1147,3.186111
2,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13.0,35.0,0.0,2.0,2014-01-25,0.0,1.0,4.0,0.0,0.0,2.0,1131,1131,3.141667
3,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,22.0,29.0,2.0,4.0,2014-01-26,0.0,0.0,3.0,0.0,0.0,3.0,1130,1130,3.138889
4,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1.0,29.0,2.0,2.0,2014-01-29,0.0,1.0,3.0,0.0,0.0,1.0,1127,1127,3.130556
5,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,13.0,21.0,0.0,3.0,2014-02-12,0.0,0.0,2.0,0.0,0.0,2.0,1113,1113,3.091667
6,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,13.0,29.0,2.0,2.0,2014-02-28,0.0,1.0,3.0,0.0,0.0,2.0,1097,1097,3.047222
7,dfLS2/Pom6O3iUpo+mf/K1Bc6g+7LFKzoNf+zlJtDoQ=,1.0,29.0,2.0,2.0,2014-03-23,0.0,1.0,3.0,0.0,0.0,1.0,1074,1074,2.983333
8,a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=,13.0,32.0,2.0,2.0,2014-03-24,0.0,1.0,3.0,0.0,0.0,2.0,1073,1073,2.980556
9,f91n3lDipDjRtAVNgMGnTsxP0VrzDsfB0Hdth12gqQE=,1.0,29.0,2.0,2.0,2014-04-20,0.0,1.0,3.0,0.0,0.0,1.0,1046,1046,2.905556


In [55]:
df.drop('registration_init_time',axis = 1, inplace =True)

In [56]:
df

,msno,city,bd,gender,registered_via,is_registerd_via_4,is_registerd_via_7,bd_zone,is_living_in_city-21,is_living_in_city-2,city_zone,how_long_has_benn_a_memmbership_of_kkbox,how_long_has_benn_a_memmbership_of_kkbox_days,how_long_has_benn_a_memmbership_of_kkbox_years
0,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13.0,18.0,1.0,4.0,0.0,0.0,1.0,0.0,0.0,2.0,1160,1160,3.222222
1,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,29.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1147,1147,3.186111
2,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13.0,35.0,0.0,2.0,0.0,1.0,4.0,0.0,0.0,2.0,1131,1131,3.141667
3,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,22.0,29.0,2.0,4.0,0.0,0.0,3.0,0.0,0.0,3.0,1130,1130,3.138889
4,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1.0,29.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1127,1127,3.130556
5,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,13.0,21.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,2.0,1113,1113,3.091667
6,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,13.0,29.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,2.0,1097,1097,3.047222
7,dfLS2/Pom6O3iUpo+mf/K1Bc6g+7LFKzoNf+zlJtDoQ=,1.0,29.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1074,1074,2.983333
8,a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=,13.0,32.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,2.0,1073,1073,2.980556
9,f91n3lDipDjRtAVNgMGnTsxP0VrzDsfB0Hdth12gqQE=,1.0,29.0,2.0,2.0,0.0,1.0,3.0,0.0,0.0,1.0,1046,1046,2.905556


In [60]:
T = 0
folder = 'trainW-'+str(T)
train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))[['msno']] # we do not need is_churn
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(train)

df = pd.merge(train, demographics, on = 'msno',how = 'left')
if T == 0:
    now_time = datetime.strptime('2017-03-01', '%Y-%m-%d') 
elif T == 1:
    now_time = datetime.strptime('2017-02-01', '%Y-%m-%d')
elif T == 2:
    now_time = datetime.strptime('2017-01-01', '%Y-%m-%d')
else:
    now_time = datetime.strptime('2017-04-01', '%Y-%m-%d')

df['how_long_has_benn_a_memmbership_of_kkbox_days'] = [(now_time - datetime.utcfromtimestamp(r_i_t.tolist()/1e9)).days if pd.notnull(r_i_t) else -1 for r_i_t in df.registration_init_time.values]
df['how_long_has_benn_a_memmbership_of_kkbox_years'] = [h_days/360 if h_days != -1 else -1 for h_days in df.how_long_has_benn_a_memmbership_of_kkbox_days.values]
df.drop('registration_init_time',axis = 1, inplace =True)
#==============================================================================
print('reduce memory')
#==============================================================================
utils.reduce_memory(df)
# write


100%|██████████| 1/1 [00:00<00:00, 3246.37it/s]

reduce memory



100%|██████████| 13/13 [00:00<00:00, 2344.09it/s]

reduce memory


In [61]:
df

,msno,city,bd,gender,registered_via,is_registerd_via_4,is_registerd_via_7,bd_zone,is_living_in_city-21,is_living_in_city-2,city_zone,how_long_has_benn_a_memmbership_of_kkbox_days,how_long_has_benn_a_memmbership_of_kkbox_years
0,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13,18,1,4,0,0,1,0,0,2,1160,3.222222
1,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,29,2,2,0,1,3,0,0,1,1147,3.186111
2,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13,35,0,2,0,1,4,0,0,2,1131,3.141667
3,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,22,29,2,4,0,0,3,0,0,3,1130,3.138889
4,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,1,29,2,2,0,1,3,0,0,1,1127,3.130556
5,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,13,21,0,3,0,0,2,0,0,2,1113,3.091667
6,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,13,29,2,2,0,1,3,0,0,2,1097,3.047222
7,dfLS2/Pom6O3iUpo+mf/K1Bc6g+7LFKzoNf+zlJtDoQ=,1,29,2,2,0,1,3,0,0,1,1074,2.983333
8,a7AtvhlY8KnKZGpiVe+4HhFerqTEgmno3x7Rc7YGwzw=,13,32,2,2,0,1,3,0,0,2,1073,2.980556
9,f91n3lDipDjRtAVNgMGnTsxP0VrzDsfB0Hdth12gqQE=,1,29,2,2,0,1,3,0,0,1,1046,2.905555
